In [1]:
import tensorflow as tf
import numpy as np
import csv

In [2]:
from google.colab import drive
drive.mount('/content/drive')

with open("/content/drive/MyDrive/data/eng-ind.csv") as f:
  csv_reader = csv.reader(f)
  next(csv_reader)

  input_data = list()
  target_data = list()

  for row in csv_reader:
    input_data.append(row[0])
    target_data.append(row[1])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
BUFFER_SIZE = len(input_data)
BATCH_SIZE = 64

dataset = tf.data.Dataset.from_tensor_slices((input_data, target_data)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)

In [4]:
for inp_batch, targ_batch in dataset.take(1):
  print(inp_batch)
  print()
  print(targ_batch)
  break

tf.Tensor(
[b"It's a good question." b"I don't have a bicycle."
 b"If you don't want to miss the train, you'd better hurry."
 b'We kept on working for hours without eating anything.'
 b'There were no roses in the garden.' b"Don't move!"
 b'I have seen that girl before.' b'Could you turn down the radio?'
 b'He has a good brain.' b'He has a hundred dollars at most.'
 b'Tom is a bartender.' b"I still haven't learned to drive a car."
 b'He seems sick.' b'Tom put on my shirt.'
 b'Several gardeners look after the azaleas in the park.'
 b"We're already planning to do that." b'Tom died from lack of oxygen.'
 b'Tom started playing the violin when he was thirteen.'
 b'Good traditions should be preserved.' b'I think my exam went well.'
 b"I can't bend my right arm." b"It's me." b"What's a UFO?"
 b"Tom says he's busy today." b"I don't have anything to hide."
 b'Tom cried.' b'Tom is nervous.' b'What do you usually do on Sundays?'
 b'You must follow school rules.' b'I ate three pieces of cake.'
 b'I